# Lista Prodotti DOP - Unione Europea

La lista dei prodotti DOP nell'Unione Europea è aggiornata dal Dipartimento per l'Agricoltura e lo Sviluppo Rurale.
http://ec.europa.eu/agriculture/quality/door/list.html?locale=it

In [ ]:
http://ec.europa.eu/agriculture/quality/door/Denominations.xls?locale